In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.environ.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="tavily"

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

In [3]:
tool.invoke({"query": "What happened at the last wimbledon"})

[{'url': 'https://www.nbcnews.com/news/sports/andy-murray-wimbledon-tennis-singles-draw-rcna159912',
  'content': "By Patrick Smith. LONDON — Andy Murray, one of the last decade's most successful male tennis players, has pulled out of the singles tournament at what is almost certain to be his last Wimbledon ..."},
 {'url': 'https://www.theguardian.com/sport/live/2023/jul/16/wimbledon-mens-singles-final-2023-carlos-alcaraz-v-novak-djokovic-live?page=with:block-64b3ff568f08df28470056bf',
  'content': 'Carlos Alcaraz recovered from a set down to topple Djokovic 1-6, 7-6(6), 6-1, 3-6, 6-4 and win his first Wimbledon title in a battle for the ages'},
 {'url': 'https://www.cnn.com/2024/07/05/sport/andy-murray-wimbledon-farewell-ceremony-spt-intl/index.html',
  'content': "It was an emotional night for three-time grand slam champion Andy Murray on Thursday, as the 37-year-old's Wimbledon farewell began with doubles defeat."},
 {'url': 'https://www.cnn.com/2024/07/09/sport/novak-djokovic-wimbl

In [4]:
from langchain_core.tools import tool
@tool
def TavilySearchAPIWrapperTool(query:str): 
    """Tool to perform internet search  using Tavily search engine"""
    tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)
    return tool.invoke({"query": query})


In [5]:
from langchain_openai import ChatOpenAI

tools = [TavilySearchAPIWrapperTool]

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [6]:
from langchain import hub
from langchain.agents import create_tool_calling_agent

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

agent = create_tool_calling_agent(llm, tools, prompt)

c:\Code\Github\LangChain-Basics\venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [7]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
query = "Tell me latest hot news?"

In [9]:
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `TavilySearchAPIWrapperTool` with `{'query': 'latest news'}`


[{'url': 'https://www.cnn.com/', 'content': 'View the latest news and breaking news today for U.S., world, weather, entertainment, politics and health at CNN.com.'}, {'url': 'https://apnews.com/', 'content': 'In a political shift to the far right, anti-Islam populist Geert Wilders wins big in Dutch election\nEurope’s far-right populists buoyed by Wilders’ win in Netherlands, hoping the best is yet to come\nDaniel Noboa is sworn in as Ecuador’s president, inheriting the leadership of a country on edge\nOn the cusp of climate talks, UN chief Guterres visits crucial Antarctica\nBUSINESS\nOpenAI brings back Sam Altman as CEO just days after his firing unleashed chaos\nThis week’s turmoil with ChatGPT-maker OpenAI has heightened trust concerns in the AI world\nTo save the climate, the oil and gas sector must slash planet-warming operations, report says\nArgentina’s labor leaders

{'input': 'Tell me latest hot news?',
 'output': "Here are some of the latest hot news stories:\n\n1. **Geert Wilders' Victory in Dutch Elections**: In a political shift to the far right, anti-Islam populist Geert Wilders has won big in the Dutch election, potentially reshaping the political landscape in Europe.\n\n2. **Ecuador's New President**: Daniel Noboa has been sworn in as Ecuador’s president, taking over leadership amid significant challenges facing the country.\n\n3. **OpenAI Turmoil**: Sam Altman has been reinstated as CEO of OpenAI just days after his firing caused chaos within the company, raising concerns about trust in the AI industry.\n\n4. **Gaza Crisis**: A hostage deal and truce between Israel and Hamas is set to begin, highlighting ongoing tensions and humanitarian issues in the region.\n\n5. **Health Alerts**: There is a deadly listeria outbreak linked to peaches and nectarines, and researchers are investigating a mysterious respiratory illness affecting dogs.\n\n6.